In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import os
import ssl
import codecs
import traceback
import urllib.parse
import pandas as pd
import csv
from bs4 import BeautifulSoup
import sys
import time

In [2]:
# 社内環境からアクセスするためのおまじない
'''
os.environ["HTTP_PROXY"] = "http://${ProxyサーバのIPアドレス}:${ProxyサーバのPort番号}/"
os.environ["HTTPS_PROXY"] = "http://${ProxyサーバのIPアドレス}:${ProxyサーバのPort番号}/"
'''
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
output_dirname = 'sankei_news'
if not os.path.exists(output_dirname):
    os.mkdir(output_dirname)

In [4]:
# ニュース検索（産経新聞）のURL
URL = 'https://www.sankei.com/search/?kw={0:}&p={1:}'

def get_links(query=None):
    # Selenium用オプション
    op = Options()
    op.add_argument('--disable-gpu')
    op.add_argument('--disable-extensions')
    op.add_argument('--headless')
    op.add_argument('--ignore-certificate-errors')
    op.add_argument('--ignore-ssl-errors')

    # オプション指定して起動
    driver = webdriver.Chrome(options=op)
    # 明示的な待機
    # 一度設定するとfind_element等の処理時に、要素が見つかるまで指定時間の間繰り返し探索するように
    driver.implicitly_wait(10)

    try:
        for page in range(1, sys.maxsize):
            # HTTPリクエストを発行
            print(URL.format(urllib.parse.quote_plus(query, encoding='utf-8'), page))
            driver.get(URL.format(urllib.parse.quote_plus(query, encoding='utf-8'), page))
            time.sleep(5)

            # HTMLドキュメントからBeautifulSoupを初期化
            soup = BeautifulSoup(driver.page_source, "html.parser")
            #soup = BeautifulSoup(html.content, "lxml")
            # 検索結果のニュース件数表示部分を検索し、件数（Int）を取得
            notfound = soup.find('p', class_='margin-bottom-lg')
            #print(notfound)
            if notfound:
                # 全て取得済み
                print(notfound.text)
                break
            # 検索結果のニュース部分を検索し、全てを取得
            soupNewsQuery = soup.find('ul', class_='sk-searchresults storycard-feed').find_all('li')
            #print(len(soupNewsQuery))

            articles = []
            for idx, soupNews in enumerate(soupNewsQuery):
                article = {
                    'url': soupNews.find('h2', class_='headline color-gray-80 margin-bottom').a.get('href'),
                    'title': soupNews.find('h2', class_='headline color-gray-80 margin-bottom').a.text,
                    'datetime': soupNews.find('div', class_='under-headline margin-bottom').time.get('datetime'),
                    'summary': soupNews.find('p', class_='article-preview color-gray-60').text
                }
                #print(article)
                articles.append(article)
                #print('%2d: %s' %(idx + 1, soupNews.find('span', class_='SearchResult_Headline').em.span.text))

            df = pd.DataFrame(articles)
            # DataFrameからCSVファイルを生成
            # encoding='sjis'だとJupyterLab（CSVTable）上で表示不可なことに注意
            #df.to_csv('sankei_news_query_%s.csv' %query, encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
            df.to_csv('sankei_news_query_%s.csv' %query, encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL, mode='a', header=False) # 追記の場合
            time.sleep(5)
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細
        print(traceback.format_exc().rstrip())
        print('=====')
    else:
        print('Exceptionが起きてなければ実行')
    finally:
        print('Exceptionが起きても実行')
        driver.close()

In [5]:
get_links('コロナ 政府 緊急事態宣言 解除 飲食店 酒')

https://www.sankei.com/search/?kw=%E3%82%B3%E3%83%AD%E3%83%8A+%E6%94%BF%E5%BA%9C+%E7%B7%8A%E6%80%A5%E4%BA%8B%E6%85%8B%E5%AE%A3%E8%A8%80+%E8%A7%A3%E9%99%A4+%E9%A3%B2%E9%A3%9F%E5%BA%97+%E9%85%92&p=1
https://www.sankei.com/search/?kw=%E3%82%B3%E3%83%AD%E3%83%8A+%E6%94%BF%E5%BA%9C+%E7%B7%8A%E6%80%A5%E4%BA%8B%E6%85%8B%E5%AE%A3%E8%A8%80+%E8%A7%A3%E9%99%A4+%E9%A3%B2%E9%A3%9F%E5%BA%97+%E9%85%92&p=2
https://www.sankei.com/search/?kw=%E3%82%B3%E3%83%AD%E3%83%8A+%E6%94%BF%E5%BA%9C+%E7%B7%8A%E6%80%A5%E4%BA%8B%E6%85%8B%E5%AE%A3%E8%A8%80+%E8%A7%A3%E9%99%A4+%E9%A3%B2%E9%A3%9F%E5%BA%97+%E9%85%92&p=3
コロナ 政府 緊急事態宣言 解除 飲食店 酒に一致する情報は見つかりませんでした
Exceptionが起きてなければ実行
Exceptionが起きても実行


In [6]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('sankei_news_query_コロナ 政府 緊急事態宣言 解除 飲食店 酒.csv', encoding='utf-8', names=['url', 'title', 'datetime', 'summary'])
df.head()

,url,title,datetime,summary
0,/article/20211012-VVZLUHSK6BI4PIHKZFL32XTCUY/,【デジタル・ウオー】敗戦か逆襲か㊥北のスパイ峻別 個人情報の国家管理受け入れる韓国,2021-10-11T22:00:00Z,「本当にお待たせしました」。東京都北区のイタリア料理店「赤バル レッツェ赤羽店」の店長、寄木...
1,/article/20211010-FVJFP3VOQVKAFOAHOLXZI5AE6U/,駆け込み殺到「認証店神話」に潜む感染リスク,2021-10-10T10:00:00Z,新型コロナウイルス対策の緊急事態宣言が解除され、酒類が提供できるようになるなどの緩和が進む。...
2,/article/20210928-VWZS57ETCRJIVNIHJTD25NBIE4/,関西でも制限緩和へ、滋賀は全面解除,2021-09-28T12:43:23.291Z,新型コロナウイルス対策の緊急事態宣言の解除決定を受け、宣言発令中の関西４府県も２８日、１０月...
3,/article/20210928-SYMWPZ6X4JKDRAYF3LVNLDUI3A/,飲食店、観光地…酒提供可に喜びも「街は変わった」不安,2021-09-28T10:38:38.922Z,新型コロナウイルスの感染状況が改善し１９都道府県を対象にした緊急事態宣言の解除が正式に決定し...
4,/article/20210928-RNNZHO6MEZITTJHOZVGPSU7JPQ/,緊急事態解除で認証店は午後９時まで、酒提供認める 分科会了承,2021-09-28T03:11:48.212Z,政府は２８日午前、新型コロナウイルスの基本的対処方針分科会を開き、コロナ特別措置法に基づき１...
